In [8]:
from datetime import datetime
import random, time, math
from copy import deepcopy, copy
import decimal

class NQueensBoard:
    def __init__(self, n_queens=8):
        self.n_queens = n_queens
        self.reset()

    def reset(self):
        # Initialize queens on random positions
        self.queen_positions = [-1 for _ in range(self.n_queens)]

        for row in range(self.n_queens):
            self.queen_positions[row] = random.randint(0, self.n_queens - 1)

    def calculate_threats(self):
        threat_count = 0

        for i in range(self.n_queens):
            for j in range(i + 1, self.n_queens):
                if self.queen_positions[i] == self.queen_positions[j] or \
                   abs(i - j) == abs(self.queen_positions[i] - self.queen_positions[j]):
                    threat_count += 1

        return threat_count

    @staticmethod
    def calculate_threats_with_positions(queen_positions):
        threat_count = 0
        n_queens = len(queen_positions)

        for i in range(n_queens):
            for j in range(i + 1, n_queens):
                if queen_positions[i] == queen_positions[j] or \
                   abs(i - j) == abs(queen_positions[i] - queen_positions[j]):
                    threat_count += 1

        return threat_count

    @staticmethod
    def format_board(queen_positions):
        board_representation = ""
        for row, col in enumerate(queen_positions):
            board_representation += "(%s, %s)\n" % (row, col)
        return board_representation

    def get_lower_cost_board(self):
        temp_queens = self.queen_positions[:]
        min_cost = self.calculate_threats()

        for i in range(self.n_queens):
            # Try moving the queen in each row and check the cost
            temp_queens[i] = (temp_queens[i] + 1) % self.n_queens
            new_cost = self.calculate_threats_with_positions(temp_queens)
            if new_cost < min_cost:
                min_cost = new_cost
                best_queens = temp_queens[:]

        return best_queens, min_cost

    def __str__(self):
        return self.format_board(self.queen_positions)


class SimulatedAnnealingSolver:
    def __init__(self, board):
        self.elapsed_time = 0
        self.board = board
        self.temperature = 4000
        self.cooling_rate = 0.99
        self.start_time = datetime.now()

    def run(self):
        current_board = self.board
        current_queens = current_board.queen_positions[:]
        solution_found = False

        for iteration in range(170000):
            self.temperature *= self.cooling_rate
            current_board.reset()
            new_queens = current_board.queen_positions[:]
            delta_w = NQueensBoard.calculate_threats_with_positions(new_queens) - NQueensBoard.calculate_threats_with_positions(current_queens)

            # Calculate the acceptance probability
            # exp_factor = decimal.Decimal(math.e) ** (-decimal.Decimal(delta_w) / decimal.Decimal(self.temperature))
            try:
                exp_factor = decimal.Decimal(math.e) ** (-decimal.Decimal(delta_w) / decimal.Decimal(self.temperature))
            except decimal.Overflow:
                exp_factor = 0.0

            # Accept the new configuration if it's better or based on probability
            if delta_w <= 0 or random.uniform(0, 1) < exp_factor:
                current_queens = new_queens[:]

            # Check if a solution is found
            if NQueensBoard.calculate_threats_with_positions(current_queens) == 0:
                print("Solution found:")
                print(NQueensBoard.format_board(current_queens))
                self.elapsed_time = self.get_elapsed_time()
                print(f"Success! Elapsed time: {self.elapsed_time} ms")
                solution_found = True
                break

        if not solution_found:
            self.elapsed_time = self.get_elapsed_time()
            print(f"Unsuccessful, Elapsed time: {self.elapsed_time} ms")

        return self.elapsed_time

    def get_elapsed_time(self):
        end_time = datetime.now()
        elapsed_time = (end_time - self.start_time).microseconds / 1000
        return elapsed_time


if __name__ == '__main__':
    board = NQueensBoard()
    print("Initial board configuration:")
    print(board)
    solver = SimulatedAnnealingSolver(board)
    solver.run()


Initial board configuration:
(0, 2)
(1, 6)
(2, 1)
(3, 0)
(4, 4)
(5, 7)
(6, 6)
(7, 1)

Solution found:
(0, 3)
(1, 5)
(2, 7)
(3, 2)
(4, 0)
(5, 6)
(6, 4)
(7, 1)

Success! Elapsed time: 916.666 ms
